In [12]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy.table import Table
import GCRCatalogs
from GCR import GCRQuery
import statistics
import os, sys
import healpy as hp
from tqdm.auto import tqdm, trange
import scipy.stats
import FoFCatalogMatching
import pandas as pd
from astropy.coordinates import SkyCoord
from scipy.stats import pearsonr
import warnings
from numpy import random
import time
# from dask4in2p3.dask4in2p3 import Dask4in2p3
# import dask
import pickle

In [13]:
warnings.filterwarnings('ignore')

# cosmoDC2, DC2objects and Filters

In [14]:
mag_filters = [
    'mag_i_cModel < 24.5', 'mag_i_cModel > 18',
    (np.isfinite, 'mag_i_cModel')
]
truth_mag_filters = (['mag_i<24.5','mag_i>18',(np.isfinite, 'mag_i')]) #larger cut to ensure matching between faintest objects/galaxies

In [15]:
cosmodc2 = GCRCatalogs.load_catalog('cosmoDC2_v1.1.4_image') #truth catalog
dc2 = GCRCatalogs.load_catalog('dc2_object_run2.2i_dr6_with_addons') #object catalog

In [16]:
object_data = (dc2.get_quantities(['ra', 'dec', 'mag_i_cModel','mag_r_cModel', 'mag_z_cModel', 'photoz_mean', 'x', 'y', 'Ixx_pixel', 'Iyy_pixel', 'Ixy_pixel', 'photoz_mode', 'blendedness', 'patch', 'snr_i_cModel'],
                                 filters=['extendedness>0', 'clean']+mag_filters,
                                 native_filters=['tract==4026']))

In [17]:
eps = 10/3600 # 10 arcsec
max_ra = np.nanmax(object_data['ra']) + eps
min_ra = np.nanmin(object_data['ra']) - eps
max_dec = np.nanmax(object_data['dec']) + eps
min_dec = np.nanmin(object_data['dec']) - eps
pos_filters = [f'ra >= {min_ra}',f'ra <= {max_ra}', f'dec >= {min_dec}', f'dec <= {max_dec}']

vertices = hp.ang2vec(np.array([min_ra, max_ra, max_ra, min_ra]),
                      np.array([min_dec, min_dec, max_dec, max_dec]), lonlat=True)
ipix = hp.query_polygon(32, vertices, inclusive=True)
healpix_filter = GCRQuery((lambda h: np.isin(h, ipix, True), "healpix_pixel"))

In [18]:
quantities = ['galaxy_id', 'ra', 'dec', 'mag_i', 'mag_r', 'mag_z', 'redshift', 'size_true', 'size_minor_true', 'position_angle_true', 'halo_id', 'halo_mass']
truth_data = (cosmodc2.get_quantities(quantities, filters=truth_mag_filters+pos_filters, 
                                      native_filters=healpix_filter))

In [19]:
stars_data = dc2.get_quantities(['ra', 'dec', 'mag_i_cModel','mag_r_cModel', 'mag_z_cModel', 'photoz_mean', 'x', 'y', 'Ixx_pixel', 'Iyy_pixel', 'Ixy_pixel', 'photoz_mode', 'blendedness'],
                                 filters=['extendedness==0', 'clean']+mag_filters,
                                 native_filters=['tract==4026'])

In [20]:
print("number of galaxies =", len(truth_data['ra']))
print("number of objects =", len(object_data['ra']))
print("number of stars =", len(stars_data['ra']))

number of galaxies = 208721
number of objects = 219425
number of stars = 13860


In [21]:
output = open('/sps/lsst/users/mramel/data/Catalogs_multiple_tracts/truth/truth_data_4026_friendly_mag.pkl', 'wb')
pickle.dump(truth_data, output)

output2 = open('/sps/lsst/users/mramel/data/Catalogs_multiple_tracts/object/object_data_4026_friendly_mag.pkl', 'wb')
pickle.dump(object_data, output2)

output3 = open('/sps/lsst/users/mramel/data/Catalogs_multiple_tracts/star_data_4026_friendly_mag.pkl', 'wb')
pickle.dump(stars_data, output3)

# FoF algorithm

In [22]:
my_linking_length = 2.

In [23]:
results = FoFCatalogMatching.match(catalog_dict={'object':pd.DataFrame(object_data), 
                                                 'galaxy':pd.DataFrame(truth_data)}, linking_lengths=my_linking_length)

In [24]:
np.save('/sps/lsst/users/mramel/data/FoF_multiple_tracts/ll_12/tract_4026_friendly_ll2_mag.npy', results)

In [25]:
print(results)

row_index catalog_key group_id
--------- ----------- --------
        0      object        0
   193443      galaxy        0
        1      object        1
    69400      galaxy        1
        2      object        2
   188711      galaxy        2
        3      object        3
   113332      galaxy        3
        4      object        4
    99750      galaxy        4
      ...         ...      ...
   208689      galaxy   224558
   208693      galaxy   224559
   208694      galaxy   224560
   208698      galaxy   224561
   208700      galaxy   224562
   208704      galaxy   224563
   208706      galaxy   224564
   208708      galaxy   224565
   208709      galaxy   224566
   208714      galaxy   224567
   208717      galaxy   224568
Length = 428146 rows


In [26]:
print(type(results))

<class 'astropy.table.table.Table'>


In [27]:
results = results.to_pandas()

# Recover groups

In [28]:
results['is_object'] = (results['catalog_key']=='object')

In [29]:
res = results.drop('catalog_key', axis=1)

In [30]:

groups = []

for group_id, rows in res.groupby('group_id'):
    id_gal = list(rows['row_index'][~rows['is_object']])
    id_obj = list(rows['row_index'][rows['is_object']])
    groups.append([id_gal, id_obj])


In [31]:
np.save("groups_ll_2_mag.npy", groups)

In [32]:
print(groups[:10])

[[[193443], [0]], [[69400], [1]], [[188711], [2]], [[113332], [3]], [[99750], [4]], [[99751], [5]], [[99149], [6]], [[61932], [7]], [[108306], [8]], [[79982], [9]]]
